IA & Data science (LU3IN0226) -- 2020-2021
--------
*&copy; Equipe pédagogique: Vincent Guigue, Christophe Marsala, Edoardo Sarti, Olivier Schwander.*


## Projet 2021

### Préparation du notebook


<font size="+1" color="RED">**[Q]**</font> **Indiquer dans la boîte ci-dessous vos noms et prénoms :**

JDAY Achraf

TALEB Rida

<font color="RED" size="+1">**[Q]**</font> **Renommer ce fichier ipython**

Tout en haut de cette page, cliquer sur <tt>projet-2021</tt> et rajouter à la suite de <tt>projet-2021</tt> les noms des membres du binômes séparés par un tiret.

Par exemple, pour le binôme Luke Skywalker et Han Solo, le nom de fichier devient `projet2021-Skywalker-Solo`

Penser à sauvegarder fréquemment le fichier en cours de travail :
- soit en cliquant sur l'icône "disquette"
- soit par la combinaison de touches [Ctrl]-S

## Données

Les données vous sont fournies sur le moodle. 
Ces données sont fournies sur Kaggle, ce sont les données *Google Play Store Apps* accessibles à l'adresse https://www.kaggle.com/lava18/google-play-store-apps.

Il est indispensable de lire en détail la page Kaggle pour comprendre à quoi ces données correspondent.

Le compte-rendu a fournir le jour de la dernière séance de TDTME de votre groupe doit comporter:
- un fichier PDF qui correspond à un poster sur lequel sont expliqués les différents problèmes traités, la façon dont ils ont été traités, et les résultats obtenus.
- un notebook par problème traité, vous pouvez traiter autant de problème que vous le souhaitez. Le problème étudié doit être décrit précisément et vous devez impérativement suivre le format ci-dessous.

Bien entendu, le tout sera mis dans un fichier archive (tar.gz ou zip exclusivement) et déposé sur le site Moodle.


Format à suivre:

## Partie 1 - Description du problème

#### Utilisation et analyse d'une base de données de reviews numérique sur les applications mobile du Google PlayStore afin de prédire si l'application réussira ou non. Au premier lieu l'application réussi si elle a un nombre d'installations très élevé ( >100000), après nous modifions cette contrainte pour que ca soit un problème multiclasses dans les cas qui suivent.
#### Cette problèmatique est interessante surtout pour ceux qui souhaitent développer une app pour le PlayStore mais veulent vérifier si elle sera un succés avant en se basant sur ses caractéristiques ce qui permettra au développeur d'avoir un pre-insight sur le développement de son app et une longueur d'avance sur ses concurrents. 

## Partie 2 - Modèle

#### Nous utilisons des modèles d'apprentissage supervisé que nous avons codé par nous meme, le Perceptron, le ADALINE Analytique, le MultiOOA et finalement les arbres de décisions numérique, tout en changont de paramètres dans plusieurs cas en split ainsi que en cross-validation strat après normalisation.

## Partie 3 - Code

In [ ]:
# Importation des librairies standards:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import plotly.express as px

# Importation de la librairie iads
import iads as iads

# importation de Classifiers
from iads import Classifiers as classif

# importation de utils
from iads import utils as ut

In [ ]:
%load_ext autoreload
%autoreload 2


### LECTURE DE L'ENSEMBLE DE DONNÉES

In [ ]:
Application_data=pd.read_csv('GoogleApps/googleplaystore.csv')

### NETTOYAGE DE L'ENSEMBLE DE DONNÉES

#### L'ensemble de données aura des valeurs redondantes comme NaN, ou certaines colonnes n'auront pas de valeur du tout, certaines colonnes auront des valeurs sans rapport, certaines auront des caractères spéciaux qui ne peuvent pas être introduits dans notre modèle d'apprentissage automatique. Ces incohérences seront donc résolues dans cette section.

In [ ]:
Application_data.head()

In [ ]:
Application_data.columns

In [ ]:
Application_data.describe()

### Déplaçons-nous de gauche à droite dans les colonnes de l'ensemble de données, nous commençons par la colonne "RATING", et nous nous déplaçons jusqu'à la colonne "PRICE", puisque ce sont des colonnes numériques et des caractéristiques nécessaires pour notre modèle. Nous allons effectuer le processus suivant sur chacune de ces colonnes :
#### 1)- Vérification de toutes les valeurs uniques dans la colonne.
#### 2)- S'il y a des valeurs uniques non liées qui ne sont pas significatives, elles seront remplacées.
#### 3)- La vérification des valeurs nulles est effectuée sur chaque colonne numérique, si des entrées nulles sont trouvées, elles sont remplacées par les valeurs moyennes.
#### 4)- Les valeurs dans les colonnes contiennent certains caractères spéciaux, qui doivent être supprimés afin d'effectuer des agrégations, ils seront donc supprimés comme "+" et "," dans la colonne Installs, "M", "Varies with Device" et "k" dans la colonne Size etc.
#### 5)- Les colonnes qui sont de type objet seront converties en leurs équivalents numériques pour l'analyse et les tendances.
#### 6)- Un filtrage final sera effectué pour s'assurer qu'il n'y a aucune incohérence dans aucune colonne qui pourrait affecter la performance de notre modèle.


#### Nettoyage de la colonne "RATING" 

In [ ]:
#Vérifier s'il y a des valeurs nulles dans la colonne Ratings.
nullcheck_ratings=pd.isnull(Application_data["Rating"])
Application_data[nullcheck_ratings]

In [ ]:
#Remplacer les valeurs NaN par la valeur moyenne de l'évaluation
Application_data["Rating"].fillna(value=Application_data["Rating"].mean(),inplace=True)
Application_data["Rating"]

In [ ]:
# En vérifiant les valeurs uniques dans la colonne Classement, nous constatons qu'il y a une valeur incohérente de 19.
Application_data["Rating"].unique()

In [ ]:
# Remplacer la valeur incohérente par la valeur moyenne des évaluations
Application_data["Rating"].replace(19.,4.1,inplace=True)

#### Aucun caractère spécial n'a été trouvé dans la colonne des évaluations, l'étape 4 n'est donc pas nécessaire. De plus, le type de données de la colonne ratings est déjà float, donc pas besoin de conversion. Donc maintenant notre colonne Rating est prête pour l'analyse.

#### Nettoyage de la colonne "REVIEWS" 

In [ ]:
# En vérifiant les valeurs uniques de la colonne nombre de critiques, nous constatons qu'il n'y a pas de valeurs non liées.
len(Application_data["Reviews"].unique())

In [ ]:
# En vérifiant les valeurs nulles de la colonne nombre de critiques, nous constatons qu'il n'y a pas de valeurs nulles.
nullcheck_reviews=pd.isnull(Application_data["Reviews"])
Application_data[nullcheck_reviews]

In [ ]:
# En vérifiant les caractères spéciaux qui pourraient empêcher la conversion numérique, 3.0M est remplacé par sa valeur réelle pour rendre les données cohérentes.
Application_data["Reviews"].replace("3.0M","3000000",inplace=True)

In [ ]:
# Enfin, nous convertissons le type de données de la colonne Reviews du type Object (String) au type Numeric (float ou int).
Application_data["Reviews"]=pd.to_numeric(Application_data["Reviews"])

#### Toutes les étapes ont été réalisées pour la colonne de reviews et elle est également prête pour l'analyse.

#### Nettoyage de la colonne "SIZE" 

In [ ]:
# En vérifiant les valeurs uniques de la colonne Size, on observe qu'elle a des valeurs accompagnées de M,k et "Varies with device".
Application_data["Size"].unique()

In [ ]:
# Remplacer le champ "Varies with device" par des entrées NaN, afin de pouvoir les remplacer ultérieurement par des valeurs moyennes.
Application_data['Size'].replace('Varies with device', np.nan, inplace = True )
Application_data['Size'].replace('1,000+', np.nan, inplace = True )


In [ ]:
# Vérifier les valeurs nulles que nous allons trouver, puisque dans la ligne ci-dessus nous avons ajouté quelques valeurs nulles.
nullcheck_size=pd.isnull(Application_data["Size"])
Application_data[nullcheck_size]


#### Nous devons maintenant remplacer les valeurs NaN par la taille moyenne de toutes les applications, mais nous ne pouvons pas calculer la moyenne car notre colonne est de type Object String, nous devons donc la convertir en type numérique. De plus, nous devons supprimer "M", "k" des valeurs de la colonne, car nous ne pouvons pas les convertir en numérique sans manipuler ces symboles spéciaux.

In [ ]:
Application_data.Size = (Application_data.Size.replace(r'[kM]+$','', regex=True).astype(float) *
                         Application_data.Size.str.extract(r'[\d\.]+([kM]+)', expand=False).fillna(1).replace(['k','M'], [10**3, 10**6]).astype(int))

In [ ]:
# Enfin, remplacer les valeurs NaN par la valeur moyenne.
Application_data["Size"].fillna(value="21516530",inplace=True)

In [ ]:
# Après avoir supprimé les caractères spéciaux, convertissons-le en type de données numériques pour trouver la valeur moyenne.
Application_data["Size"]=pd.to_numeric(Application_data["Size"])

#### Ici, nous avons terminé le nettoyage de la colonne Taille en suivant les 6 étapes qui étaient requises, puisque cette colonne était très peu nettoyée.

#### Le nettoyage de la colonne "INSTALL" 

In [ ]:
# En vérifiant les valeurs uniques de la colonne Installs, nous observons qu'il existe un type appelé "free", qui est incohérent et non numérique, il doit donc être remplacé.
Application_data["Installs"].unique()

#### Nous devons supprimer le mot "gratuit" du nombre moyen d'installations des applications, mais pour calculer la moyenne, nous devons supprimer les signes "+" et "," des valeurs. Après les avoir supprimés, nous devrons les convertir en type numérique, puis nous pourrons calculer la moyenne et enfin substituer la valeur moyenne à la place de "Free".

In [ ]:
# Suppression du symbole "+" pour rendre la colonne numérique.
Application_data["Installs"]=Application_data["Installs"].map(lambda x: x.rstrip('+'))

In [ ]:
# Enlever le "," des chiffres pour faciliter la tâche.
Application_data["Installs"]=Application_data["Installs"].str.replace(",","")

In [ ]:
# Aucune entrée nulle n'a été trouvée dans cette colonne.
nullcheck_installs=pd.isnull(Application_data["Installs"])
Application_data[nullcheck_installs]

In [ ]:
# Remplacer la valeur incohérente de l'étiquette par la valeur moyenne de la colonne.
Application_data["Installs"].replace("Free","15462910",inplace=True)

In [ ]:
# Convertir le type de données en type numérique pour l'analyse
Application_data["Installs"]=pd.to_numeric(Application_data["Installs"])

#### De cette façon, nous avons rendu notre colonne Installs prête pour l'analyse en suivant à nouveau les 6 étapes.

#### Nettoyage de la colonne "TYPE"

In [ ]:
# En vérifiant les valeurs uniques, nous avons trouvé nan et 0 qui doivent être remplacés par Free.
Application_data["Type"].unique()

In [ ]:
# Remplacement de 0 par Free
Application_data["Type"].replace("0","Free",inplace=True)

In [ ]:
# Remplir les valeurs manquantes avec Free, puisque la plupart des applications sont gratuites sur Google play.
Application_data["Type"].fillna(value="Free",inplace=True)

In [ ]:
# Ajouter les colonnes fictives pour cela, afin qu'elles puissent contribuer à notre modèle.
dummy_type=pd.get_dummies(Application_data["Type"])

In [ ]:
#Concaténation des colonnes fictives avec le cadre de données principal.
Application_data=pd.concat([Application_data,dummy_type],axis=1)

In [ ]:
# Finalement, on laisse tomber la colonne de type.
Application_data.drop(["Type"],axis=1,inplace=True)

In [ ]:
Application_data.head()

#### De cette façon, nous avons supprimé la colonne catégorique Type, utilisé des colonnes fictives pour rendre notre espace de caractéristiques plus précis.

#### Nettoyage de la colonne "PRICE" 


In [ ]:
# En vérifiant les valeurs uniques, nous constatons que "Everyone" est une valeur incohérente qui doit être supprimée.
Application_data["Price"].unique()

#### Ici, pour obtenir la moyenne des valeurs, le type de données de la colonne doit être numérique et pour cela, nous devons supprimer le symbole du dollar des valeurs et abandonner la ligne de tout le monde, car elle contient des données redondantes qui compromettront les performances de notre modèle.

In [ ]:
# Suppression du symbole du dollar
Application_data["Price"]=Application_data["Price"].map(lambda x: x.lstrip('$'))

In [ ]:
# Suppression de la valeur de la ligne non essentielle.
Application_data.drop(Application_data[Application_data["Price"] == "Everyone"].index, inplace=True)

In [ ]:
# En vérifiant qu'il n'y a pas de valeurs nulles trouvées
nullcheck_Prices=pd.isnull(Application_data["Price"])
Application_data[nullcheck_Prices]

In [ ]:
# Enfin convertir en type numérique pour l'analyse
Application_data["Price"]=pd.to_numeric(Application_data["Price"])

#### Nous avons nettoyé la colonne Prix en suivant les 6 étapes selon les besoins, maintenant cette colonne est prête pour l'analyse.

#### Nettoyage de la colonne "CATEGORY" 

In [ ]:
# En vérifiant les valeurs uniques, nous avons trouvé 
Application_data["Category"].unique()

In [ ]:
Application_data["Category"].replace("1.9","MISCELLANEOUS",inplace=True)

In [ ]:
# Vérification des valeurs nulles, aucune valeur nulle n'a été trouvée pour cette colonne.
nullcheck=pd.isnull(Application_data["Category"])
Application_data[nullcheck]

#### Pour cette colonne, nous allons procéder à l'encodage des étiquettes et non des mannequins, car en faisant des mannequins, nous ajouterons trop de colonnes supplémentaires à notre matrice de caractéristiques, ce qui n'est pas nécessaire. L'encodage des étiquettes est donc effectué en fournissant des valeurs numériques à chaque catégorie d'application.

In [ ]:
# Importation de la bibliothèque requise
from sklearn.preprocessing import LabelEncoder

In [ ]:
# Instanciation de l'encodeur
labelencoder2 = LabelEncoder()

In [ ]:
#Encodage de la colonne Category en utilisant scikit learn
Application_data['Categories_encoded'] = labelencoder2.fit_transform(Application_data['Category'])

In [ ]:
# enfin, on laisse tomber la colonne type, puisqu'elle est déjà fractionnée.
Application_data.drop(["Category"],axis=1,inplace=True)

In [ ]:
Application_data.head()

#### Nettoyage de la colonne "CONTENT RATING" 

#### Pour cette colonne catégorielle également, nous effectuons l'encodage des étiquettes comme nous l'avons fait pour la colonne Catégorie.

In [ ]:
Application_data["Content Rating"].unique()

In [ ]:
nullcheck_contentrating=pd.isnull(Application_data["Content Rating"])
Application_data[nullcheck_contentrating]

In [ ]:
labelencoder = LabelEncoder()

In [ ]:
Application_data['Content_Rating_encoded'] = labelencoder.fit_transform(Application_data['Content Rating'])

In [ ]:
Application_data.drop(["Content Rating"],axis=1,inplace=True)

In [ ]:
Application_data.head()

In [ ]:
# Vérifier les types de données des colonnes pour s'assurer que nous avons réussi à rassembler toutes les colonnes numériques.
Application_data.dtypes

In [ ]:
# Trouver la moyenne de toutes les colonnes numériques
Application_data.mean()

### ANALYSE DES DONNÉES

#### Vous trouverez ci-dessous une analyse complète des diverses relations entre les caractéristiques de nos données. Cette analyse est nécessaire pour que nous puissions comprendre quelles sont les caractéristiques qui joueront un rôle important dans la prédiction du nombre d'installations d'une application.

In [ ]:
sns.pairplot(Application_data)

#### Ici, un diagramme de paires est montré entre toutes les colonnes numériques des données. Cela donne un haut niveau d'intuition entre les relations entre les différentes caractéristiques. Tout d'abord, des histogrammes seront dessinés pour toutes les colonnes numériques afin de connaître leur nombre et leur distribution. Plotly est utilisé ici pour les représentations graphiques.

In [ ]:
colorassigned=Application_data["Rating"]
fig = px.histogram(Application_data, x="Rating", marginal="rug",
                   hover_data=Application_data.columns,nbins=30,color=colorassigned)
fig.show()

#### Le graphique ci-dessus est un histogramme, qui montre la distribution des évaluations de diverses applications androïdes. L'histogramme est divisé en couleurs en fonction des valeurs de l'évaluation. L'échelle de couleurs est indiquée sur le côté droit. Le nombre de notes 4.1 est maximal (1474) comme on peut le constater en survolant le graphique. De plus, le nombre de notes augmente uniformément de 3,4 (128) à 4,1 (1474), puis augmente et diminue à nouveau. Cela signifie que la plupart des applications sur Google Play ont des notes comprises entre 4 et 4,5.


In [ ]:
fig = px.histogram(Application_data, x="Reviews", marginal="rug",
                   hover_data=Application_data.columns,nbins=30)
fig.show()

#### Voici un histogramme qui montre la distribution du nombre d'avis pour chaque application. Il est clairement visible que 90% des applications sur Google Play Store ont moins de 5 millions d'avis. 138 applications ont des avis entre 5 et 10 millions. Seules 47 applications androïdes ont des avis entre 10 et 15 millions. La majorité des applications ont donc moins de 5 millions d'avis.

In [ ]:
colorassigned=Application_data["Size"]
fig = px.histogram(Application_data, x="Size", marginal="rug",
                   hover_data=Application_data.columns,nbins=30,color=colorassigned)
fig.show()

#### Le graphique ci-dessus est un histogramme, qui montre la distribution de la taille de diverses applications androïdes. On peut observer que la plupart des applications ont une taille inférieure, puisque lorsque la taille augmente sur l'axe des x, les barres deviennent de plus en plus courtes, ce qui signifie que le nombre de ces types d'applications diminue. Nous avons donc plus d'applications sur Google playstore qui sont de petite taille que de grandes applications. La plupart des applications ont une taille d'environ 21,5 Mo.


In [ ]:
colorassigned=Application_data["Installs"]
fig = px.histogram(Application_data, x="Installs", marginal="rug",
                   hover_data=Application_data.columns,nbins=30,color=colorassigned)
fig.show()

#### Le graphique ci-dessus montre le nombre d'installations d'applications androïdes. On peut observer que la majorité des applications ont moins de 10 millions d'installations. De plus, il n'y a que 58 applications qui ont plus d'un milliard d'installations sur Google play.

In [ ]:
colorassigned=Application_data["Price"]
fig = px.histogram(Application_data, x="Price", marginal="rug",
                   hover_data=Application_data.columns,nbins=30,color=colorassigned)
fig.show()

#### Cet histogramme montre la répartition des prix de diverses applications androïdes sur Google play. La majorité des applications sont gratuites. Il y a 12 applications androïdes qui sont les plus chères, coûtant 400 dollars. 

## Nous avons ainsi terminé l'analyse individuelle de toutes les colonnes numériques de notre ensemble de données. Maintenant, nous allons trouver la relation entre chaque colonne pour l'analyser en profondeur. L'étape suivante est la suivante :
### 1)- Calculer la valeur de corrélation et dessiner une carte thermique pour connaître la corrélation entre les différentes colonnes.
### 2)- Une fois que nous avons trouvé la corrélation, nous savons quelles colonnes s'influencent les unes les autres, puis nous commençons à tracer les colonnes par paire en fonction de leurs valeurs de corrélation. Si la corrélation est négative ou très faible, il n'y a aucun intérêt à tracer ces colonnes.
### 3)- Après le tracé, nous ajustons une ligne de régression linéaire à nos points de données. Plus la valeur de corrélation est élevée, meilleure est la ligne d'ajustement que nous obtenons.


In [ ]:
# Calculating the Correlation and plotting the heatmap to know the relations.
cors=Application_data.corr()
fig = px.imshow(cors,labels=dict(color="Pearson Correlation"), x=['Rating', 'Reviews', 'Size', 'Installs', 'Price','Paid','Free','Content_Rating_encoded','Categories_encoded'],
                y=['Rating', 'Reviews', 'Size','Installs','Price','Paid','Free','Content_Rating_encoded','Categories_encoded'])
fig.show()

## Les conclusions suivantes peuvent être tirées de cette carte thermique :
### VALEUR DE CORRÉLATION CARACTÉRISTIQUES IMPLIQUÉES VERDICT

-0,020 Prix par rapport au classement Aucune corrélation

### -0,009 Prix par rapport aux avis Pas de corrélation

### -0.022 Prix vs Taille Pas de corrélation

### 0,011 Prix par rapport aux installations Pas de corrélation

### 0,051 Installations par rapport au classement Aucune corrélation

### 0,643 Installes vs critiques Grande corrélation

### 0,082 Installes vs Taille Pas de corrélation

### -0.011 Installes vs Prix Pas de corrélation

### 0,074 Taille par rapport au classement Aucune corrélation

### 0,128 Taille par rapport aux évaluations Corrélation très faible

### 0,082 Taille par rapport aux installations Aucune corrélation

### -0.022 Taille vs Prix Pas de corrélation

### 0.067 Critiques vs Classement Pas de corrélation



## Nous ne tracerons que les relations dont la valeur de corrélation est supérieure à 0,1, les autres n'ayant aucune corrélation, le tracé ne sera pas fructueux. 

In [ ]:
# Tracer un diagramme de dispersion avec une ligne d'ajustement entre les installations et les évaluations, ces deux éléments ont la corrélation la plus élevée entre eux.
from scipy.stats import pearsonr 
corryu,_ =pearsonr(Application_data["Installs"],Application_data["Reviews"])
colorassigned=Application_data["Reviews"]
fig = px.scatter(Application_data, x="Installs", y="Reviews",trendline="ols",color=colorassigned)
fig.show()
print("Pearson Correlation: %.3f" % corryu)
print("P-value: %.8f" % _)

#### On observe que nous avons un bon ajustement aux points de données, puisque la corrélation entre ces 2 colonnes est significative. Comme on peut le voir, le nombre de critiques augmente avec le nombre d'installations, ce qui est logique, puisque si l'utilisateur a installé l'application, il est le seul à pouvoir donner son avis. Sans l'utilisation d'une application, il est impossible de donner des avis. Si nous obtenons un nouveau point de données, nous pouvons prédire son nombre d'installations sur la base du nombre d'évaluations. En survolant la ligne rouge, on peut voir l'équation de la ligne droite. En survolant chaque point de données, on obtient le nombre d'installations et d'évaluations à ce point.

In [ ]:
# Tracer un nuage de points avec une ligne d'ajustement entre le classement et les critiques, ces deux éléments ont une corrélation très faible entre eux. 
from scipy.stats import pearsonr 
corryu,_ =pearsonr(Application_data["Rating"],Application_data["Reviews"])
colorassigned=Application_data["Reviews"]
fig = px.scatter(Application_data, x="Rating", y="Reviews",trendline="ols",color=colorassigned)
fig.show()
print("Pearson Correlation: %.3f" % corryu)
print("P-value: %.8f" % _)

#### Comme on peut l'observer sur ce graphique, on constate que les applications qui ont des notes comprises entre 4 et 4,7 ont un nombre maximum d'avis. Cependant, nous ne pouvons pas dire que plus les notes augmentent, plus le nombre d'avis augmente, cela se produit juste pour une plage particulière de 4 à 4,7 où les avis augmentent en même temps que les notes, mais avant 4 et après 4,7, la tendance est différente. On observe qu'après une note de 4,7, le nombre d'avis a diminué, c'est-à-dire que le nombre d'applications ayant reçu un avis a diminué. Les applications ayant 5 étoiles n'ont que 4 commentaires. Cependant, les applications ayant une note inférieure à 4 ont été évaluées par de nombreux utilisateurs.

In [ ]:
# Tracer un nuage de points avec une ligne d'ajustement entre la taille et les critiques, ces deux éléments ont une corrélation très faible entre eux. 
from scipy.stats import pearsonr 
corryu,_ =pearsonr(Application_data["Size"],Application_data["Reviews"])
colorassigned=Application_data["Reviews"]
fig = px.scatter(Application_data, x="Size", y="Reviews",trendline="ols",color=colorassigned)
fig.show()
print("Pearson Correlation: %.3f" % corryu)
print("P-value: %.8f" % _)

#### Il n'y a pas de tendance générale observée dans ce graphique, car il y a très peu de corrélation observée dans ces deux colonnes. Il y a des applications de 21 Mo qui obtiennent 80 millions d'avis, et il y a des applications de taille plus importante, comme 98 Mo, qui obtiennent 45 millions d'avis. Il n'y a donc pas de tendance observée ici.

In [ ]:
from scipy.stats import pearsonr 
corryu,_ =pearsonr(Application_data["Installs"],Application_data["Categories_encoded"])
colorassigned=Application_data["Categories_encoded"]
fig = px.scatter(Application_data, x="Installs", y="Categories_encoded",trendline="ols",color=colorassigned)
fig.show()
print("Pearson Correlation: %.3f" % corryu)
print("P-value: %.8f" % _)

## Partie 4 - Protocole expérimental

## Cas 1: deux labels

### Feature Selection et Split des datas

In [ ]:
len(Application_data["Categories_encoded"].unique())

In [ ]:
Application_data["Installs"].max()

In [ ]:
def label(df):
    x = df['Installs']
    if x < 100000:
        return -1
    else:
        return 1

Application_data['Label'] = Application_data.apply(lambda df: label(df), axis=1)
Application_data.head()

In [ ]:
Xa=Application_data[["Reviews","Size","Rating","Price","Paid","Free","Categories_encoded","Content_Rating_encoded"]]
ya=Application_data["Label"].values
print(Xa)
x1=Xa.values

In [ ]:
from sklearn import preprocessing
Xa = preprocessing.StandardScaler().fit(Xa).transform(Xa)

In [ ]:
print(Xa)

In [ ]:
from sklearn.model_selection import train_test_split
Xa_train, Xa_test, ya_train, ya_test = train_test_split(Xa, ya, test_size=0.2, random_state=42)

In [ ]:
print(Xa_train)

In [ ]:
print(ya_train)

### Perceptron

In [ ]:
learning_rate = 0.001

perceptron = classif.ClassifierPerceptron(8,learning_rate)

In [ ]:
perceptron.train(Xa_train,ya_train)
a=perceptron.accuracy(Xa_train,ya_train)


In [ ]:
for i in range(0,5):
    print(i+1,": (",ya_train[i],") --> ",perceptron.predict(Xa_train[i,:]), "(",perceptron.score(Xa_train[i,:]),")")

In [ ]:
a1=perceptron.accuracy(Xa_test,ya_test)


In [ ]:
for i in range(0,5):
    print(i+1,": (",ya_test[i],") --> ",perceptron.predict(Xa_test[i,:]), "(",perceptron.score(Xa_test[i,:]),")")

### ADALINE

In [ ]:
Adaline = classif.ClassifierADALINE2(8,100)

In [ ]:
Adaline.train(Xa_train,ya_train)
b=Adaline.accuracy(Xa_train,ya_train)


In [ ]:
for i in range(0,5):
    print(i+1,": (",ya_train[i],") --> ",Adaline.predict(Xa_train[i,:]), "(",Adaline.score(Xa_train[i,:]),")")

In [ ]:
b1=Adaline.accuracy(Xa_test,ya_test)


In [ ]:
for i in range(0,5):
    print(i+1,": (",ya_test[i],") --> ",Adaline.predict(Xa_test[i,:]), "(",Adaline.score(Xa_test[i,:]),")")

### Arbre de décisions

In [ ]:
noms = ["Reviews","Size","Rating","Price","Paid","Free","Categories_encoded","Content_Rating_encoded"]

In [ ]:
arbreA = classif.CAD(8, 0, noms)

In [ ]:
arbreA.train(x1,ya)

In [ ]:
import graphviz as gv
gr_arbreA = gv.Digraph(format='png')
arbreA.affiche(gr_arbreA)


In [ ]:
arbreA1 = classif.CAD(8, 0.25, noms)

In [ ]:
arbreA1.train(x1,ya)

In [ ]:
import graphviz as gv
gr_arbreA1 = gv.Digraph(format='png')
arbreA1.affiche(gr_arbreA1)


## Cas 2: multi labels

### Feature Selection et Split des datas

In [ ]:
def label(df):
    x = df['Installs']
    if x < 100000:
        return -1
    elif x < 10000000:
        return 1
    else:
        return 2

Application_data['Label'] = Application_data.apply(lambda df: label(df), axis=1)
Application_data.head()

In [ ]:
Xb=Application_data[["Reviews","Size","Rating","Price","Paid","Free","Categories_encoded","Content_Rating_encoded"]]
yb=Application_data["Label"].values
print(Xb)
x1=Xb.values

In [ ]:
from sklearn import preprocessing
Xb = preprocessing.StandardScaler().fit(Xb).transform(Xb)

In [ ]:
from sklearn.model_selection import train_test_split
Xb_train, Xb_test, yb_train, yb_test = train_test_split(Xb, yb, test_size=0.2, random_state=42)

### Perceptron

In [ ]:

learning_rate = 0.001

perceptron = classif.ClassifierPerceptron(8,learning_rate)

perceptmulti = classif.ClassifierMultiOAA(perceptron)

In [ ]:
perceptmulti.train(Xb_train,yb_train)
c=perceptmulti.accuracy(Xb_train,yb_train)


In [ ]:
for i in range(0,5):
    print(i+1,": (",yb_train[i],") --> ",perceptmulti.predict(Xb_train[i,:]), "(",perceptmulti.score(Xb_train[i,:]),")")

In [ ]:
c1=perceptmulti.accuracy(Xb_test,yb_test)


In [ ]:
for i in range(0,5):
    print(i+1,": (",yb_test[i],") --> ",perceptmulti.predict(Xb_test[i,:]), "(",perceptmulti.score(Xb_test[i,:]),")")

### ADALINE

In [ ]:
Adaline = classif.ClassifierADALINE2(8,100)
AdalineMulti = classif.ClassifierMultiOAA(Adaline)

In [ ]:
AdalineMulti.train(Xb_train,yb_train)
d=AdalineMulti.accuracy(Xb_train,yb_train)


In [ ]:
for i in range(0,5):
    print(i+1,": (",yb_train[i],") --> ",AdalineMulti.predict(Xb_train[i,:]), "(",AdalineMulti.score(Xb_train[i,:]),")")

In [ ]:
d1=AdalineMulti.accuracy(Xb_test,yb_test)


In [ ]:
for i in range(0,5):
    print(i+1,": (",yb_test[i],") --> ",AdalineMulti.predict(Xb_test[i,:]), "(",AdalineMulti.score(Xb_test[i,:]),")")

### Arbre de décisions

In [ ]:
arbreB = classif.CAD(8, 0, noms)

In [ ]:
arbreB.train(x1,yb)

In [ ]:
import graphviz as gv
gr_arbreB = gv.Digraph(format='png')
arbreB.affiche(gr_arbreB)


In [ ]:
arbreB1 = classif.CAD(8, 0.25, noms)

In [ ]:
arbreB1.train(x1,yb)

In [ ]:
import graphviz as gv
gr_arbreB1 = gv.Digraph(format='png')
arbreB1.affiche(gr_arbreB1)


## Cas 3: Multi labels + less features

### Feature Selection et Split des datas

In [ ]:
def label(df):
    x = df['Installs']
    if x < 100000:
        return -1
    elif x < 10000000:
        return 1
    else:
        return 2

Application_data['Label'] = Application_data.apply(lambda df: label(df), axis=1)
Application_data.head()

In [ ]:
Xc=Application_data[["Reviews","Price","Rating","Categories_encoded"]]
yc=Application_data["Label"].values
print(Xc)
x1 = Xc.values

In [ ]:
from sklearn import preprocessing
Xc = preprocessing.StandardScaler().fit(Xc).transform(Xc)

In [ ]:
from sklearn.model_selection import train_test_split
Xc_train, Xc_test, yc_train, yc_test = train_test_split(Xc, yc, test_size=0.2, random_state=42)

### Perceptron

In [ ]:

learning_rate = 0.001

perceptron = classif.ClassifierPerceptron(4,learning_rate)

perceptmulti = classif.ClassifierMultiOAA(perceptron)

In [ ]:
perceptmulti.train(Xc_train,yc_train)
e=perceptmulti.accuracy(Xc_train,yc_train)


In [ ]:
for i in range(0,5):
    print(i+1,": (",yc_train[i],") --> ",perceptmulti.predict(Xc_train[i,:]), "(",perceptmulti.score(Xc_train[i,:]),")")

In [ ]:
e1=perceptmulti.accuracy(Xc_test,yc_test)


In [ ]:
for i in range(0,5):
    print(i+1,": (",yc_test[i],") --> ",perceptmulti.predict(Xc_test[i,:]), "(",perceptmulti.score(Xc_test[i,:]),")")

### ADALINE

In [ ]:
Adaline = classif.ClassifierADALINE2(4,100)
AdalineMulti = classif.ClassifierMultiOAA(Adaline)

In [ ]:
AdalineMulti.train(Xc_train,yc_train)
f=AdalineMulti.accuracy(Xc_train,yc_train)


In [ ]:
for i in range(0,5):
    print(i+1,": (",yc_train[i],") --> ",AdalineMulti.predict(Xc_train[i,:]), "(",AdalineMulti.score(Xc_train[i,:]),")")

In [ ]:
f1=AdalineMulti.accuracy(Xc_test,yc_test)


In [ ]:
for i in range(0,5):
    print(i+1,": (",yc_test[i],") --> ",AdalineMulti.predict(Xc_test[i,:]), "(",AdalineMulti.score(Xc_test[i,:]),")")

### Arbre de décisions

In [ ]:
noms = ["Reviews","Price","Rating","Categories_encoded"]

In [ ]:
arbreC = classif.CAD(4, 0.0, noms)
arbreC.train(x1,yc)

In [ ]:
import graphviz as gv
gr_arbreC = gv.Digraph(format='png')
arbreC.affiche(gr_arbreC)


In [ ]:
arbreC1 = classif.CAD(4, 0.25, noms)
arbreC1.train(x1,yc)

In [ ]:
import graphviz as gv
gr_arbreC1 = gv.Digraph(format='png')
arbreC1.affiche(gr_arbreC1)


## Partie 5 - Résultats

## Cas 1: deux labels

### Perceptron

In [ ]:
print("Accuracy sur données d'apprentissage: ",a)

In [ ]:
print("Accuracy sur données de tests: ",a1)

In [ ]:
#Â Changement du learning rate : on le prend trÃ¨s grand !
learning_rate = 1e-3

#Â Graine pour les tirages alÃ©atoires :
np.random.seed(42)   # supprimer cette ligne une fois la mise au point terminÃ©e

niter = 10
perf = []

for i in range(niter):
    Xapp,Yapp,Xtest,Ytest = ut.crossval_strat(Xa, ya, niter, i)
    cl = classif.ClassifierPerceptron(8,learning_rate)
    for j in range(0,10):
        cl.train(Xapp, Yapp)
    perf.append(cl.accuracy(Xtest, Ytest))
    print("Apprentissage ",i+1,":\t"," |Yapp|= ",len(Yapp)," |Ytest|= ",len(Ytest),"\tperf= ",perf[-1])

# On transforme la liste en array numpy pour avoir les fonctions statistiques:
perf = np.array(perf)
print(f'\nRÃ©sultat global:\tmoyenne= {perf.mean():.3f}\tÃ©cart-type= {perf.std():.3f}')

In [ ]:
plt.figure()
plt.plot(perf)
plt.title("Evolution de l'apprentissage")
plt.xlabel('Itération')

In [ ]:
#Â Choix du learning rate
learning_rate = 1e-3

#Â Graine pour les tirages alÃ©atoires :
np.random.seed(42)  

#Â CrÃ©ation et entraÃ®nement du perceptron sur les donnÃ©es gÃ©nÃ©rÃ©es
#Â On utilise la mÃ©morisation de l'historique des poids comme vu en TME 4
perceptronV1 = classif.ClassifierPerceptron(8, learning_rate, history=True)

# on rÃ©alise 10 appels de train:
for i in range(0,10):
    perceptronV1.train(Xa_train,ya_train)

# rÃ©cupÃ©ration de l'Ã©volution des w au cours de l'apprentissage 
allw = np.array(perceptronV1.allw) # si allw est sous forme de liste

#Â TracÃ© de l'Ã©volution des w:
plt.figure()
plt.plot(allw[:,0]) #Â premiÃ¨re coordonnÃ©e du vecteur poids: w1
plt.plot(allw[:,1]) #Â deuxiÃ¨me coordonnÃ©e du vecteur poids: w2
plt.title('Evolution des w au cours des itérations du perceptron')
plt.xlabel('iterations')
plt.legend(['w1','w2'])

#Â Performance de ce classifieur:
print("Accuracy du perceptron (", learning_rate,"): ",perceptronV1.accuracy(Xa_test,ya_test)) 
print("Vecteur de poids final trouvé: ", perceptronV1.getW())


### ADALINE

In [ ]:
print("Accuracy sur données d'apprentissage: ",b)

In [ ]:
print("Accuracy sur données de tests: ",b1)

### Arbre de décisions 

In [ ]:
gr_arbreA

In [ ]:
gr_arbreA1

## Cas 2: Multi labels

### Perceptron

In [ ]:
print("Accuracy sur données d'apprentissage: ",c)

In [ ]:
print("Accuracy sur données de tests: ",c1)

In [ ]:
#Â On va utiliser un chronomÃ¨tre pour avoir le temps d'exÃ©cution :
import timeit
#Â Changement du learning rate : on le prend trÃ¨s grand !
learning_rate = 1e-3

#Â Graine pour les tirages alÃ©atoires :
np.random.seed(42)   # supprimer cette ligne une fois la mise au point terminÃ©e

niter = 10
perf = []

tic = timeit.default_timer() # heure de dÃ©part
for i in range(niter):
    Xapp,Yapp,Xtest,Ytest = ut.crossval_strat(Xb, yb, niter, i)
    cl = classif.ClassifierMultiOAA(classif.ClassifierPerceptron(8,learning_rate))
    cl.train(Xapp, Yapp)
    perf.append(cl.accuracy(Xtest, Ytest))
    print("Apprentissage ",i+1,":\t"," |Yapp|= ",len(Yapp)," |Ytest|= ",len(Ytest),"\tperf= ",perf[-1])
toc = timeit.default_timer() # heure d'arrivÃ©e

# On transforme la liste en array numpy pour avoir les fonctions statistiques:
perf = np.array(perf)

print(f'\nTemps mis: --> {toc-tic:.5f} secondes')
print(f'RÃ©sultat global:\tmoyenne= {perf.mean():.3f}\tÃ©cart-type= {perf.std():.3f}')

In [ ]:
plt.figure()
plt.plot(perf)
plt.title("Evolution de l'apprentissage")
plt.xlabel('Itération')

### ADALINE

In [ ]:
print("Accuracy sur données d'apprentissage: ",d)

In [ ]:
print("Accuracy sur données de tests: ",d1)

### Arbre de décisions

In [ ]:
gr_arbreB

In [ ]:
gr_arbreB1

## Cas 3: Multi labels + less features

### Perceptron

In [ ]:
print("Accuracy sur données d'apprentissage: ",e)

In [ ]:
print("Accuracy sur données d'apprentissage: ",e1)

In [ ]:
#Â Changement du learning rate : on le prend trÃ¨s grand !
learning_rate = 1e-3

#Â Graine pour les tirages alÃ©atoires :
np.random.seed(42)   # supprimer cette ligne une fois la mise au point terminÃ©e

niter = 10 
perf = []

tic = timeit.default_timer() # heure de dÃ©part
for i in range(niter):
    Xapp,Yapp,Xtest,Ytest = ut.crossval(Xc, yc, niter, i)
    cl = classif.ClassifierMultiOAA(classif.ClassifierPerceptron(4,learning_rate))
    cl.train(Xapp, Yapp)
    perf.append(cl.accuracy(Xtest, Ytest))
    print("Apprentissage ",i+1,":\t"," |Yapp|= ",len(Yapp)," |Ytest|= ",len(Ytest),"\tperf= ",perf[-1])
toc = timeit.default_timer() # heure d'arrivÃ©e

# On transforme la liste en array numpy pour avoir les fonctions statistiques:
perf = np.array(perf)

print(f'\nTemps mis: --> {toc-tic:.5f} secondes')
print(f'RÃ©sultat global:\tmoyenne= {perf.mean():.3f}\tÃ©cart-type= {perf.std():.3f}')

In [ ]:
plt.figure()
plt.plot(perf)
plt.title("Evolution de l'apprentissage")
plt.xlabel('Itération')

### ADALINE

In [ ]:
print("Accuracy sur données d'apprentissage: ",f)

In [ ]:
print("Accuracy sur données de tests: ",f1)

### Arbre de décisions

In [ ]:
gr_arbreC

In [ ]:
gr_arbreC1

### Comparaison entre les Adaline et Perceptron des 3 cas

In [ ]:
data = {'Classifieur': ['Perceptron cas1','Adaline cas1','Perceptron cas2','Adaline cas2','Perceptron cas3','Adaline cas3'], 'Apprentissage Accuracy': [a/100, b/100, c, d, e, f], 'Tests Accuracy': [a1/100, b1/100, c1, d1, e1, f1]}
dfdata = pd.DataFrame(data)
print(dfdata)

## Partie 6 - Analyse

#### D'après les statistiques sur les modèles et les résultats nous pouvons clairement constater que le problème binaire marche le meilleur mais pour des raisons de réalisme nous allons regarder que le problème multiclasses et alors seulement le 2eme cas vu que d'après nos experimentations dans le 3eme cas le moins de dimensions/features on a le moins l'accuracy qu'on aura ce qui est logique.
#### Dans ce cas, le Perceptron est mieux que le Adaline dans les tests sur les données d'apprentissage ainsi que les données de tests, l'abre de décisions nous montre aussi des résultats logique (Exemple: réussite de l'app s'il y a un très grand nombre de reviews dans une catégorie populaire).


#### Conclusion: Résolution de problème réussi, nous avons réussi a prédire a 60% pret le succées d'une application mobile a partir de ses caractéristiques.  